In [75]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [76]:
%%R
library(mltools)
library(data.table)
library(dplyr)
library(ggplot2)
library(ggiraph)
library('mclust')

IN_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_validation/output/'
OUT_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_validation/output_ARI/'

gnn_data <- read.csv(paste0(IN_DIR, 'gnn_predictions4.tsv'), sep = "\t", row.names = NULL)
resnet_data <- read.csv(paste0(IN_DIR, 'resnet_predictions.tsv'), sep = "\t", row.names = NULL)

In [77]:
%%R
str(resnet_data)

'data.frame':	713 obs. of  5 variables:
 $ X..sample_id: chr  "SRR10108618" "ERR2588648" "SRR2984107" "ERR1121528" ...
 $ issues      : chr  "Liver" "Brain" "Brain" "Liver" ...
 $ target      : int  6 1 1 6 6 7 12 8 0 6 ...
 $ prediction  : int  6 1 1 6 6 7 12 1 0 6 ...
 $ confidences : num  0.98 0.756 0.91 0.986 0.984 ...
'data.frame':	713 obs. of  5 variables:
 $ X..sample_id: chr  "SRR10108618" "ERR2588648" "SRR2984107" "ERR1121528" ...
 $ tissues     : chr  "Liver" "Brain" "Brain" "Liver" ...
 $ target      : int  6 1 1 6 6 7 12 8 0 6 ...
 $ prediction  : int  6 1 1 6 6 7 12 1 4 6 ...
 $ confidences : num  0.992 0.995 0.994 0.999 0.995 ...


In [78]:
%%R
print(unique(gnn_data$issues))
print(gnn_data)

 [1] "Liver"     "Brain"     "Lung"      "Testes"    "Muscle"    "Adipose"  
 [7] "Intestine" "Pancreas"  "Eye"       "Kidney"    "Ovary"     "Skin"     
[13] "Heart"    
    X..sample_id    issues target prediction confidences
1    SRR10108618     Liver      6          6   0.9799147
2     ERR2588648     Brain      1          1   0.7559812
3     SRR2984107     Brain      1          1   0.9099771
4     ERR1121528     Liver      6          6   0.9859283
5     SRR6032301     Liver      6          6   0.9838080
6     SRR4216970      Lung      7          7   0.9163777
7     SRR6784823    Testes     12         12   0.9649419
8     SRR3355764    Muscle      8          1   0.3637656
9     SRR5127014   Adipose      0          0   0.8052312
10    SRR3156193     Liver      6          6   0.9809390
11    SRR7161300 Intestine      4          4   0.9984171
12    SRR7162115   Adipose      0          0   0.8849871
13    ERR2588664     Brain      1          1   0.9751480
14    ERR1121490    Muscle     

283  SRR10523569    Kidney      5          6   0.9543139
284  SRR10523534      Lung      7          5   0.6291322
285   SRR5758682    Muscle      8          8   0.8381375
286   ERR2588639     Brain      1          1   0.4139016
287   SRR7771847    Muscle      8          8   0.4262247
288   SRR6041178     Liver      6          6   0.9701179
289   SRR2993959     Liver      6          6   0.9863269
290   SRR7161281 Intestine      4          4   0.8220330
291   SRR8597226     Brain      1          1   0.9271566
292   SRR5877183       Eye      2          2   0.8035913
293   SRR3355770    Muscle      8          3   0.2993346
294  SRR10400166     Brain      1          1   0.9617954
295   SRR6269594     Brain      1          1   0.6050743
296   SRR6495798     Brain      1          1   0.9468611
297   SRR6729591     Liver      6          6   0.9830967
298   ERR2654362    Testes     12         12   0.9379440
299  SRR10400076     Brain      1          1   0.9891867
300   SRR6445075 Intestine     

569  SRR10399864     Brain      1          1   0.8605784
570   SRR6652616   Adipose      0          0   0.6715295
571   SRR9317370     Liver      6          6   0.9938226
572   SRR7162062   Adipose      0          0   0.9283942
573  SRR10031046 Intestine      4          4   0.9750456
574   SRR6221748     Liver      6          6   0.9888511
575   SRR1993673     Brain      1          2   0.7437823
576   SRR2984137     Brain      1          1   0.9203799
577   ERR2588685    Testes     12         10   0.9868841
578   SRR7127888     Ovary      9          1   0.5912541
579   SRR7072954     Liver      6          6   0.9900917
580   SRR6253451     Heart      3          3   0.4222518
581  SRR10523511     Liver      6          6   0.9735916
582   SRR7182434     Liver      6          6   0.8596130
583   SRR8874809     Heart      3          3   0.4461461
584   SRR8442756     Liver      6          6   0.9932998
585   SRR5850819    Kidney      5          6   0.9673046
586   ERR2362963 Intestine     

In [79]:
%%R
gnn_ari <- adjustedRandIndex(gnn_data$target, gnn_data$prediction)
resnet_ari <- adjustedRandIndex(resnet_data$target, resnet_data$prediction)
print(paste0("gnn_ari: ", gnn_ari))
print(paste0("resnet_ari: ", resnet_ari))

[1] "gnn_ari: 0.729016687523187"
[1] "resnet_ari: 0.709307143838519"


In [80]:
%%R
unique_tissues <- unique(c(gnn_data$target, gnn_data$prediction))
names <- unique(gnn_data$issues)
gnn_results <- data.frame(names = names, tissue = unique_tissues)

gnn_results$TP <- 0
gnn_results$TN <- 0
gnn_results$FP <- 0
gnn_results$FN <- 0

for (tissue in unique_tissues) {
  for (i in 1:nrow(gnn_data)) {
    if (gnn_data$target[i] == tissue) {
      if (gnn_data$prediction[i] == tissue) {
        gnn_results[gnn_results$tissue == tissue, "TP"] <- gnn_results[gnn_results$tissue == tissue, "TP"] + 1
      } else {
        gnn_results[gnn_results$tissue == tissue, "FN"] <- gnn_results[gnn_results$tissue == tissue, "FN"] + 1
      }
    } else {
      if (gnn_data$prediction[i] == tissue) {
        gnn_results[gnn_results$tissue == tissue, "FP"] <- gnn_results[gnn_results$tissue == tissue, "FP"] + 1
      } else {
        gnn_results[gnn_results$tissue == tissue, "TN"] <- gnn_results[gnn_results$tissue == tissue, "TN"] + 1
      }
    }
  }
}

gnn_results$POSITIVES <- gnn_results$TP + gnn_results$FN

gnn_results$MCC <- (gnn_results$TP * gnn_results$TN - gnn_results$FP * gnn_results$FN) / 
  sqrt((gnn_results$TP + gnn_results$FP) * (gnn_results$TP + gnn_results$FN) * (gnn_results$TN + gnn_results$FP) * (gnn_results$TN + gnn_results$FN))

gnn_results$TPR <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$FPR <- gnn_results$FP / (gnn_results$FP + gnn_results$TN)

gnn_results$TNR <- gnn_results$TN / (gnn_results$TN + gnn_results$FP)

gnn_results$FNR <- gnn_results$FN / (gnn_results$FN + gnn_results$TP)

gnn_results$Precision <- gnn_results$TP / (gnn_results$TP + gnn_results$FP)

gnn_results$Recall <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$F1_score <- 2 * (gnn_results$Precision * gnn_results$Recall)/(gnn_results$Precision + gnn_results$Recall)

print(gnn_results)



write.table(gnn_results, file = paste(OUT_DIR, "gnn_MCC_Result.txt"), sep="\t", row.names = FALSE, quote = FALSE)

       names tissue  TP  TN FP FN POSITIVES       MCC       TPR         FPR
1      Liver      6 198 484 18 13       211 0.8965007 0.9383886 0.035856574
2      Brain      1 140 521 30 22       162 0.7962702 0.8641975 0.054446461
3       Lung      7  21 673 10  9        30 0.6747080 0.7000000 0.014641288
4     Testes     12  23 682  1  7        30 0.8517892 0.7666667 0.001464129
5     Muscle      8  22 672  9 10        32 0.6845709 0.6875000 0.013215859
6    Adipose      0  50 646  9  8        58 0.8417514 0.8620690 0.013740458
7  Intestine      4  48 642 10 13        61 0.7894360 0.7868852 0.015337423
8   Pancreas     10  11 688 10  4        15 0.6103003 0.7333333 0.014326648
9        Eye      2  10 692  3  8        18 0.6463205 0.5555556 0.004316547
10    Kidney      5  37 649 16 11        48 0.7133625 0.7708333 0.024060150
11     Ovary      9   3 692  7 11        14 0.2410086 0.2142857 0.010014306
12      Skin     11   2 691  7 13        15 0.1585059 0.1333333 0.010028653
13     Heart

In [81]:
%%R
str(resnet_data)

'data.frame':	713 obs. of  5 variables:
 $ X..sample_id: chr  "SRR10108618" "ERR2588648" "SRR2984107" "ERR1121528" ...
 $ tissues     : chr  "Liver" "Brain" "Brain" "Liver" ...
 $ target      : int  6 1 1 6 6 7 12 8 0 6 ...
 $ prediction  : int  6 1 1 6 6 7 12 1 4 6 ...
 $ confidences : num  0.992 0.995 0.994 0.999 0.995 ...


In [82]:
%%R
unique_tissues <- unique(c(resnet_data$target, resnet_data$prediction))
names <- unique(resnet_data$tissues)
resnet_results <- data.frame(names = names, tissue = unique_tissues)

resnet_results$TP <- 0
resnet_results$TN <- 0
resnet_results$FP <- 0
resnet_results$FN <- 0

for (tissue in unique_tissues) {
  for (i in 1:nrow(resnet_data)) {
    if (resnet_data$target[i] == tissue) {
      if (resnet_data$prediction[i] == tissue) {
        resnet_results[resnet_results$tissue == tissue, "TP"] <- resnet_results[resnet_results$tissue == tissue, "TP"] + 1
      } else {
        resnet_results[resnet_results$tissue == tissue, "FN"] <- resnet_results[resnet_results$tissue == tissue, "FN"] + 1
      }
    } else {
      if (resnet_data$prediction[i] == tissue) {
        resnet_results[resnet_results$tissue == tissue, "FP"] <- resnet_results[resnet_results$tissue == tissue, "FP"] + 1
      } else {
        resnet_results[resnet_results$tissue == tissue, "TN"] <- resnet_results[resnet_results$tissue == tissue, "TN"] + 1
      }
    }
  }
}

resnet_results$POSITIVES <- resnet_results$TP + resnet_results$FN

resnet_results$MCC <- (resnet_results$TP * resnet_results$TN - resnet_results$FP * resnet_results$FN) / 
  sqrt((resnet_results$TP + resnet_results$FP) * (resnet_results$TP + resnet_results$FN) * (resnet_results$TN + resnet_results$FP) * (resnet_results$TN + resnet_results$FN))

resnet_results$TPR <- resnet_results$TP / (resnet_results$TP + resnet_results$FN)

resnet_results$FPR <- resnet_results$FP / (resnet_results$FP + resnet_results$TN)

resnet_results$TNR <- resnet_results$TN / (resnet_results$TN + resnet_results$FP)

resnet_results$FNR <- resnet_results$FN / (resnet_results$FN + resnet_results$TP)

resnet_results$Precision <- resnet_results$TP / (resnet_results$TP + resnet_results$FP)

resnet_results$Recall <- resnet_results$TP / (resnet_results$TP + resnet_results$FN)

resnet_results$F1_score <- 2 * (resnet_results$Precision * resnet_results$Recall)/(resnet_results$Precision + resnet_results$Recall)

print(resnet_results)

write.table(resnet_results, file = paste(OUT_DIR, "resnet_MCC_Result.txt"), sep="\t", row.names = FALSE, quote = FALSE)


       names tissue  TP  TN FP FN POSITIVES       MCC       TPR         FPR
1      Liver      6 194 466 36 17       211 0.8277650 0.9194313 0.071713147
2      Brain      1 147 525 26 15       162 0.8408589 0.9074074 0.047186933
3       Lung      7  15 670 13 15        30 0.4971201 0.5000000 0.019033675
4     Testes     12  20 680  3 10        30 0.7525241 0.6666667 0.004392387
5     Muscle      8  21 674  7 11        32 0.6885528 0.6562500 0.010279001
6    Adipose      0  51 645 10  7        58 0.8444645 0.8793103 0.015267176
7  Intestine      4  59 649  3  2        61 0.9555504 0.9672131 0.004601227
8   Pancreas     10  10 695  3  5        15 0.7104809 0.6666667 0.004297994
9        Eye      2  14 691  4  4        18 0.7720224 0.7777778 0.005755396
10    Kidney      5  33 654 11 15        48 0.6986953 0.6875000 0.016541353
11     Ovary      9   3 697  2 11        14 0.3515250 0.2142857 0.002861230
12      Skin     11   9 698  0  6        15 0.7712888 0.6000000 0.000000000
13     Heart

In [83]:
%%R
ari_row_gnn <- data.frame(names = paste("gnn_ARI: ", gnn_ari), tissue = NA, TP = NA, TN = NA, FP = NA, FN = NA, POSITIVES = NA, MCC = NA, TPR = NA, FPR = NA, TNR = NA, FNR = NA, Precision = NA, Recall = NA, F1_score = NA)
ari_row_resnet <- data.frame(names = paste("resnet_ARI: ", resnet_ari), tissue = NA, TP = NA, TN = NA, FP = NA, FN = NA, POSITIVES = NA, MCC = NA, TPR = NA, FPR = NA, TNR = NA, FNR = NA, Precision = NA, Recall = NA, F1_score = NA)

gnn_results <- rbind(gnn_results, ari_row_gnn)
resnet_results <- rbind(resnet_results, ari_row_resnet)

if (all(names(gnn_results) == names(resnet_results))) {
  combined_df <- rbind(gnn_results, resnet_results)
  print(combined_df)
  write.table(combined_df, file = paste(OUT_DIR, "MCC_Result.txt"), sep="\t", row.names = FALSE, quote = FALSE)
} else {
  print("Column names or orders do not match between data frames.")
}


[1] 15
[1] 15
                            names tissue  TP  TN FP FN POSITIVES       MCC
1                           Liver      6 198 484 18 13       211 0.8965007
2                           Brain      1 140 521 30 22       162 0.7962702
3                            Lung      7  21 673 10  9        30 0.6747080
4                          Testes     12  23 682  1  7        30 0.8517892
5                          Muscle      8  22 672  9 10        32 0.6845709
6                         Adipose      0  50 646  9  8        58 0.8417514
7                       Intestine      4  48 642 10 13        61 0.7894360
8                        Pancreas     10  11 688 10  4        15 0.6103003
9                             Eye      2  10 692  3  8        18 0.6463205
10                         Kidney      5  37 649 16 11        48 0.7133625
11                          Ovary      9   3 692  7 11        14 0.2410086
12                           Skin     11   2 691  7 13        15 0.1585059
13         